# База данных

In [1]:
import os
import sqlite3

In [2]:
def connect(dbfile):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    return c, conn

In [3]:
def create_db():
    dbfile = input('Type the name of the database to create (without a filename extension): ') + '.db'
    has_db = os.path.exists(dbfile)
    if has_db: 
        print('The database with this name has already existed.')
        return create_db()
    c, conn = connect(dbfile)
    print('The database "{}" is created successfully.'.format(dbfile))
    c.close()

In [4]:
# create_db()

In [5]:
def create_table():
    dbfile = input('Type the name of the database where you want to create a table\
    (without filename extension): ') + '.db'
    if not os.path.isfile(dbfile):
        print('This database does not exist. Try once again')
        create_table()
    else:
        c, conn = connect(dbfile)
        table_name = input('Type the name of the table to create: ').upper()
        columns = input('Type the names of the columns and their types as in the EXAMPLE:\
        \n"name1 text, name2 text, name3 integer"\n') 
        c.execute('CREATE TABLE {} (generated_id INTEGER PRIMARY KEY, {})'.format(table_name, columns))
        print('The table "{}" in the database "{}" is created successfully.'.format(table_name, dbfile))
        conn.commit()
        conn.close()

In [6]:
# create_table()
# c, conn = connect('realec.db')
# sql = 'DROP TABLE EXPERIMENT_1'
# c.execute(sql)
# conn.commit()
# conn.close()

Type the name of the database where you want to create a table    (without filename extension): essays
This database does not exist. Try once again
Type the name of the database where you want to create a table    (without filename extension): realec
Type the name of the table to create: main
Type the names of the columns and their types as in the EXAMPLE:        
"name1 text, name2 text, name3 integer"
name text, text text, mark integer, type integer
The table "MAIN" in the database "realec.db" is created successfully.


In [7]:
def fill_table_MAIN(dbfile, name, text, mark, _type):
    c, conn = connect(dbfile)
    info = [[name, text, mark, _type]]
    c.executemany('''INSERT INTO MAIN (generated_id, name, text, mark, type) VALUES (null, ?, ?, ?, ?)''', info)
    conn.commit()
    conn.close()
    # print('Information is filled successfully.')

In [8]:
# fill_table_MAIN('realec.db', 'test2.txt', 'This is an example', 50, 2)

In [10]:
c, conn = connect('realec.db')
c.execute("SELECT generated_id, text, type FROM MAIN WHERE MAIN.mark == 80")
result = [info for info in c.fetchall()]
print(result)

[]


In [11]:
def delete_one(dbfile, _id):
    c, conn = connect(dbfile)
    sql = 'DELETE FROM MAIN WHERE generated_id=?'
    c.execute(sql, (_id,))

In [12]:
# delete_one('realec.db', 1)

In [13]:
def delete_all_raws(dbfile):
    c, conn = connect(dbfile)
    sql = 'DELETE FROM MAIN'
    c.execute(sql)
    conn.commit()
    conn.close()

In [14]:
# delete_all_raws('realec.db')

# Добавление в базу

In [15]:
import os
from tqdm import tqdm_notebook as tqdm
import json
import re

In [16]:
def clean_mark(mark):
    mark = str(mark).replace('(overall)', '').strip()
    mark = str(mark).replace(',', '.').strip()
    mark = str(mark).replace('%', '').strip()
    if float(mark) < 10:
        mark = str(mark).replace('.', '')
        mark = str(mark).replace('.', '')
    if float(mark) > 10:
        mark = float(mark)
    if int(mark) <= 10:
        mark = int(mark) * 10
    return int(mark)

In [17]:
main_dir = './data/exam'
os.remove('files_with_json.txt')
with open('files_with_json.txt', 'a') as fw:
    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if '.json' in name and os.path.isfile(os.path.join(root, name)[:-5] + '.txt'):
                full_path = os.path.join(root, name)
                try:
                    with open(full_path) as fj:
                        data = json.load(fj)
                        if 'mark' in data.keys() and data['mark'] != '':
                            try:
                                mark = data['mark']
                                mark = clean_mark(mark)
                                fw.write(os.path.join(root, name)[:-5] + '\n')
                            except:
                                continue
                except:
                    continue

In [18]:
! wc -l < files_with_json.txt

    3446


In [19]:
def clean_path(path):
    path = path.replace('true', '')
    path = path.replace('(full)', '')
    return path

In [20]:
with open('files_with_json.txt', 'r') as fr:
    for line, path in tqdm(enumerate(fr)):
        path = path[:-1]
        with open(path + '.json') as fj:
            data = json.load(fj)
        mark = data['mark']
        mark = clean_mark(mark)
        _type = re.search('_(.)$', clean_path(path)).group(1)
        with open(path + '.txt', encoding='utf-8') as fj:
            text = fj.read()
        fill_table_MAIN('realec.db', path, text, mark, _type)

# Распределение по оценкам

In [36]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark >= 70")
result = c.fetchall()
print(result)

[(387,)]


In [37]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 70 AND MAIN.mark >= 60")
result = c.fetchall()
print(result)

[(2097,)]


In [38]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 60")
result = c.fetchall()
print(result)

[(962,)]


# Распределение по типам эссе

In [39]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.type == 1")
result = c.fetchall()
print(result)

[(1703,)]


In [40]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.type == 2")
result = c.fetchall()
print(result)

[(1742,)]


# Все вместе

In [43]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 70 AND MAIN.mark >= 60 AND MAIN.type == 1")
result = c.fetchall()
print(result)
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 70 AND MAIN.mark >= 60 AND MAIN.type == 2")
result = c.fetchall()
print(result)

[(184,)]
[(203,)]


In [44]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 70 AND MAIN.mark >= 60 AND MAIN.type == 1")
result = c.fetchall()
print(result)
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 70 AND MAIN.mark >= 60 AND MAIN.type == 2")
result = c.fetchall()
print(result)

[(1002,)]
[(1094,)]


In [45]:
c, conn = connect('realec.db')
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 60 AND MAIN.type == 1")
result = c.fetchall()
print(result)
c.execute("SELECT COUNT(*) FROM MAIN WHERE MAIN.mark < 60 AND MAIN.type == 2")
result = c.fetchall()
print(result)

[(517,)]
[(445,)]
